# DAV Use Case
# How to Work With Comprehensive Internal Model Data for Three Portfolios


### Authors:
- Christian Jonen
- Tamino Meyhöfer 
- Zoran Nikolić

Also involved in various stages of the development:
- Felix Terhag (first prototype for neural networks)
- Michelle Anell
- Helen Schneider

The main intention of this notebook is to demonstrate how the comprehensive internal model data which we provide together with the notebook can be loaded and transformed in order to train different machine learning models.

The scope of the data exceeds anything currently accessible in the open domain. Furthermore, we provide more data than is usually available in the productive environments of the insurance companies, which should enable the actuarial data scientists to reliably test their regression models and draw comparisons among them.

The code below for the training of both neural networks and polynomials serves as a minimal viable example. The resulting neural network and regression polynomial are by no means optimized. They represent simple examples, in the case of the polynomial regression we use as terms only the linear factors of the risk factors. 

The users of this Use Case are encouraged to test other hyperparameter configurations and compare the results they obtain when doing so. Furthermore, a sophisticated hyperparameter optimization together with an ensemble technique will lead to a more stable and robust result. We believe that other regression techniques ranging from more traditional to the decision tree based approaches can equally well be applied to our data.

The authors have implemented one such optimization. In the article we plan to publish soon after releasing the data and in our talks at the conferences of the German Actuarial Association we report the goodness-of-fit we have reached and compare it to the best polynomials we have fitted using the least-squares regression and an adaptive step-wise algorithm as described in Krah, Nikolić, Korn: "A Least-Squares Monte Carlo Framework in Proxy Modeling of Life Insurance Companies" (2018). These results should serve as a challenge to the actuarial/data science community. We are looking forward to approaches which will produce even better results. 


## Neural Network

In this chapter we demostrate how our data can be used to train a neural network. For the given training data a neural network model is trained with hyperparameters included in the file setup_NN.csv. As noted above, various hyperparameter optimizations as well as ensemble methods can be implemented. 



In [1]:
import numpy as np
import pandas as pd

from keras.layers import Input, Dense, LeakyReLU, Dropout
from keras.models import Model, load_model
import keras.optimizers as kOpt
from keras.regularizers import l1_l2

import pickle
import os
import time
from datetime import datetime

Using TensorFlow backend.


## Get data and scale it

In this portion of the code data are loaded for the neural network training. 

As explained in the accompanying description on the web page of the German Actuarial Association (in German) for all three portfolios we provide four sets of input and output data. Additionally, the standard errors for validation and nested data sets are provided. In the article we plan to publish later in 2020 we intend to describe the data in more detail in English. Our data consists of:

- train_input.csv
- train_result.csv
- validation_input.csv
- validation_result.csv
- nested_input.csv
- nested_result.csv
- base_input.csv
- base_result.csv    
    
The base case data are provided for the sake of completeness. This file does not include any new information, since our data is scaled in such a manner that the base case (all risk factors take value 0)  becomes 1. Technically, input files have to contain columns corresponding to input parameters and rows corresponding to samples. The first column and the first row are disregarded as they are considered headers. Output files have one column with header 'o1'. This column represents the scaled and transformed value of the economic own funds for each of the three portfolios.

The following code reads in and scales the inputs and outputs for the neural network and automatically deletes all columns containing only nans.

In [71]:
def get_data(path):

    # assert path is a string and the required files do exist
    assert type(path)==str, "path has to be a string"
    if not path[-1]=='/':
        path=path+'/'
    
    assert os.path.exists(path+'train_input.csv'), "File '"+ path+"train_input.csv' is missing"
    assert os.path.exists(path+'validation_input.csv'), "File '"+ path+"validation_input.csv' is missing"
    assert os.path.exists(path+'nested_input.csv'), "File '"+ path+"nested_input.csv' is missing"
    assert os.path.exists(path+'base_input.csv'), "File '"+ path+"base_input.csv' is missing"
    assert os.path.exists(path+'train_result.csv'), "File '"+ path+"train_result.csv' is missing"
    assert os.path.exists(path+'validation_result.csv'), "File '"+ path+"validation_result.csv' is missing"
    assert os.path.exists(path+'nested_result.csv'), "File '"+ path+"nested_result.csv' is missing"
    assert os.path.exists(path+'base_result.csv'), "File '"+ path+"base_result.csv' is missing"
    
        
    # =============================================================================
    #     Specify paths
    # =============================================================================
        
    # train paths
    path_train_res=path+'train_result.csv'
    path_train_input=path+'train_input.csv'       
            
    # validation paths  
    path_val_res=path+'validation_result.csv'
    path_val_input=path+'validation_input.csv'  

    # nested paths
    path_nest_res=path+'nested_result.csv'
    path_nest_input=path+'nested_input.csv'
        
    # base paths
    path_base_res=path+'base_result.csv'
    path_base_input=path+'base_input.csv'

    
    # =============================================================================
    #   Load data
    # =============================================================================
            
    # train set
    train_results=pd.read_csv(path_train_res,index_col=0)
    train_inputs=pd.read_csv(path_train_input,index_col=0)
    
    # validation set
    val_results=pd.read_csv(path_val_res,index_col=0)
    val_inputs=pd.read_csv(path_val_input,index_col=0)
    
    # nested set
    nest_results=pd.read_csv(path_nest_res,index_col=0)
    nest_inputs=pd.read_csv(path_nest_input,index_col=0)
    
    # base set
    base_results=pd.read_csv(path_base_res,index_col=0)
    base_results=base_results.iloc[[0],:]
    base_inputs=pd.read_csv(path_base_input,index_col=0)

    
    # =============================================================================
    #   Scale data    
    # =============================================================================
    
    # calculate scaling
    results_scale=pd.DataFrame(columns=train_results.columns)
    inputs_scale=pd.DataFrame(columns=train_inputs.columns)
    
    results_scale.loc['min']=train_results.min()
    results_scale.loc['max']=train_results.max()
    
    inputs_scale.loc['min']=train_inputs.min()
    inputs_scale.loc['max']=train_inputs.max()
    
    train_results=(train_results-results_scale.loc['min',:])/(results_scale.loc['max',:]-results_scale.loc['min',:])
    train_inputs=(train_inputs-inputs_scale.loc['min',:])/(inputs_scale.loc['max',:]-inputs_scale.loc['min',:])
    
    nest_results=(nest_results-results_scale.loc['min',:])/(results_scale.loc['max',:]-results_scale.loc['min',:])
    nest_inputs=(nest_inputs-inputs_scale.loc['min',:])/(inputs_scale.loc['max',:]-inputs_scale.loc['min',:])
    
    val_results=(val_results-results_scale.loc['min',:])/(results_scale.loc['max',:]-results_scale.loc['min',:])
    val_inputs=(val_inputs-inputs_scale.loc['min',:])/(inputs_scale.loc['max',:]-inputs_scale.loc['min',:])
    
    base_results=(base_results-results_scale.loc['min',:])/(results_scale.loc['max',:]-results_scale.loc['min',:])
    base_inputs=(base_inputs-inputs_scale.loc['min',:])/(inputs_scale.loc['max',:]-inputs_scale.loc['min',:])
    
    
    # Align order
    base_results=base_results[train_results.columns]
    base_inputs=base_inputs[train_inputs.columns]
    
    nest_results=nest_results[train_results.columns]
    nest_inputs=nest_inputs[train_inputs.columns]
    
    val_results=val_results[train_results.columns]
    val_inputs=val_inputs[train_inputs.columns]
    
    
    return train_inputs,train_results, \
            val_inputs, val_results, \
            nest_inputs, nest_results, \
            base_inputs, base_results, \
            inputs_scale, results_scale
       

def scale_back(y,scale):

    y=y*(scale.loc['max']-scale.loc['min'])+scale.loc['min']
    
    return y

In the following we establish functions for setting up the model, training it and evaluating the predictions.

Following this, we build a feedforward neural network corresponding to the given hyperparameter specifications and return a keras model. 

In [72]:
def setup_model(hidden_layers=[100,100,100,100,100],dropouts=[0.3,0.3,0.3,0.3,0.2],input_dim=15, 
                activation='LeakyReLU',loss='mean_squared_error',optimizer='adam',verbose=True,
                LReluAlpha=0.3,l1reg=0.,l2reg=0.,initializer='uniform',output_activation='tanh'):

    '''
    hidden_layers [int]      List of dimensions of the hidden layers.
    
    dropouts [float]         List of dropouts in the specific hidden layer, has to be of the same size as hidden_layers.
                            
    input_dim [int]          Input dimension.
                            
    activation               Has to be an identifier for keras activation functions,
                             LeakyReLU can be used as an additional identifier.
                                
    loss                     Loss used for keras.models.Model.compile.
    
    optimizer                Optimizer used for keras.models.Model.compile.
    
    verbose [boolean]        If False no suppresses, all outputs from this function.
                            
    LReluAlpha               Alpha for LeakyReLU activation. If another activation is used, the value is disregarded.
                        
    l1reg                    L1 regularization term.
        
    l2reg                    L2 regularization term.
    
    initializer              Keras initializer. 
    
    output_activation [str]  Activation of the output layer.
    '''

    # assert hidden_layers and dropouts have the same length
    assert len(hidden_layers)==len(dropouts),"hidden_layers and dropouts have to be of the same length"
    for i in range(len(hidden_layers)):
        assert type(hidden_layers[i])==int, "hidden dimensions have to be of type int"
    
    assert type(input_dim)==int, "input_dim has to be of type int"
    inputs = Input(shape=(input_dim,))
    
    '''first layer processing the input data'''
    if activation=='LeakyReLU':
        x=Dense(hidden_layers[0],kernel_regularizer=l1_l2(l1=l1reg, l2=l2reg),bias_regularizer=l1_l2(l1=l1reg, l2=l2reg),kernel_initializer=initializer)(inputs)
        x=LeakyReLU(alpha=LReluAlpha)(x)
        x=Dropout(dropouts[0])(x)
    else:
        x=Dense(hidden_layers[0],activation=activation,kernel_regularizer=l1_l2(l1=l1reg, l2=l2reg),bias_regularizer=l1_l2(l1=l1reg, l2=l2reg),kernel_initializer=initializer)(inputs)
        x=Dropout(dropouts[0])(x)
        
    '''hidden layers'''
    for i in range(1,len(hidden_layers)):
        if activation=='LeakyReLU':
            x = Dense(hidden_layers[i],kernel_regularizer=l1_l2(l1=l1reg, l2=l2reg),bias_regularizer=l1_l2(l1=l1reg, l2=l2reg),kernel_initializer=initializer)(x)
            x = LeakyReLU(alpha=LReluAlpha)(x)
            x = Dropout(dropouts[i])(x)
        else:
            x = Dense(hidden_layers[i],activation=activation,kernel_regularizer=l1_l2(l1=l1reg, l2=l2reg),bias_regularizer=l1_l2(l1=l1reg, l2=l2reg),kernel_initializer=initializer)(x)
            x = Dropout(dropouts[i])(x)
            
    '''output layer'''
    predictions = Dense(1, activation=output_activation)(x)
           
    # This creates a model that includes the input layer and some dense layers
    model = Model(inputs=inputs, outputs=predictions)
    
    model.compile(optimizer=optimizer,loss=loss,metrics=['mean_squared_error','mean_absolute_error'])
    
    # print if verbose
    if verbose:
        model.summary()
    
    return model

One of the important parts of our proposal for the training is to train the model on the training set but use the validation set for the early stopping. We therefore build upon the data paradigm with three types of data (training, validation, nested) used in the LSMC risk capital proxy modelling implemented in the industry. The following code saves the model and the configuration in path if a path is given.

The next function is called when the neural network model is trained.

In [73]:
def train_model(model,x,y,x_val,y_val,path=None,batchsize=128,epochs=300,early_stopping=True,early_stopping_no=200,
                results_scale=None,time_budget=np.inf, in_nest=None, in_hval=None, in_base=None,res_nest=None, 
                res_hval=None,res_base=None,val_metric='mse',result_param='o1',verbose=True):
    
    '''
    model                Model to be trained.
     
    x,y                  Training set.
    
    x_val, y_val         Validation set.
    
    path                 Saves the best and the last model in path/savemodels,
                         saves the training results in path, aborts if path already exists.
    
    batchsize [int]      Batchsize.
    
    epochs [int]         Maximum number of epochs.
    
    early_stopping [boolean]
                         Stops early if True and no improvement for [early_stopping_no] epochs.
    
    early_stopping_no [int] 
                         Stops training if it has not improved on the validation set for this number of consecutive epochs.
                    
    results_scale        The values to scale the result back to the original values in order to obtain the correct MSE.             
                    
    time_budget [float]  Stops if this time in seconds is used up.
    
    val_metric [string]  A keyword in ['mse', 'mae'], determines which metric is used for validation and hence for
                         early stopping.
    '''
    
    x=x.values
    x_val=x_val.values
    
    y=y[result_param].values
    y_val=y_val[result_param].values
    
    # check if all sets for full evaluation of all metrics are given
    if in_nest is None:
        full_eval_bool=False
    elif in_hval is None:
        full_eval_bool=False
    elif in_base is None:
        full_eval_bool=False
    elif res_nest is None:
        full_eval_bool=False
    elif res_hval is None:
        full_eval_bool=False
    elif res_base is None:
        full_eval_bool=False
    else:
        full_eval_bool=True
        

    print("%d Training samples"%len(y))
    print("%d Validation samples"%len(y_val))

    # loss on validation set
    val_mse=[];
    val_mae=[];
    
    # loss during training
    train_mse=[];
    train_mae=[];
    
    # factor to scale mse back to the mse on the original values
    if not results_scale is None:
        res_max=results_scale.loc['max',result_param]
        res_min=results_scale.loc['min',result_param]
        scale_factor=(res_max-res_min)**2
        scale_factor_abs=res_max-res_min
    else:
        scale_factor=1;
        scale_factor_abs=1;

    # if path not None, check if path/savemodels exists    
    if not path is None:
        assert type(path)==str, "Path has to be a string"
        assert not os.path.exists(path), "Path '"+ path+"' already exists"
        if not path[-1]=='/':
            path=path+'/'
        # make folder
        os.mkdir(path)
        os.mkdir(path+'savemodels')
        
    
    # save best epoch and best value for early stopping
    best_epoch=0
    best_value=np.inf
    
    start_t=time.time()
    cur_t=time.time()
    for epoch in range(epochs):
        if verbose:
            print("\n==================Epoch %d=================="%epoch)
        # train one epoch # trains the model for a given number of epochs
        hist=model.fit(x=x, y=y, validation_data=(x_val,y_val),batch_size=batchsize, epochs=epoch+1, verbose=0, callbacks=None, shuffle=True,  initial_epoch=epoch)
        # append training loss
        train_mse.append(scale_factor*hist.history['mean_squared_error'][-1])
        train_mae.append(scale_factor_abs*hist.history['mean_absolute_error'][-1])
        
        mse=scale_factor*hist.history['val_mean_squared_error'][-1]
        val_mse.append(scale_factor*hist.history['val_mean_squared_error'][-1])
        val_mae.append(scale_factor_abs*hist.history['val_mean_absolute_error'][-1])
        
        # output current evaluations
        if verbose:
            print("Train MSE:\t\t%.5g"%(scale_factor*hist.history['mean_squared_error'][-1]))
            print("Val MSE:\t\t%.5g"%(mse))
        
        
        if val_metric=='mae':
            current_best=scale_factor_abs*hist.history['val_mean_absolute_error'][-1]
        else:
            current_best=mse
        
        # check if current value is the best value encountered so far
        if current_best<best_value:
            best_value=current_best;
            best_epoch=epoch
            
            # delete old best model if there is a model saved and save the current best
            if not path is None:
                if os.path.exists(path+'savemodels/model_best.h5'):
                    os.remove(path+'savemodels/model_best.h5')
                model.save(path+'savemodels/model_best.h5')

        # check if it hasn't improved for early_stopping epochs
        elif epoch-best_epoch>=early_stopping_no:
            if early_stopping:  
                print("\nModel has not improved for %d consecutive Epochs"%early_stopping_no)
                print("Early Stopping...")
    
                break;
            
        # break if time budget is used up
        if time.time()-start_t>time_budget:
            print("\nStopped because time budget of %.1f is used up"%time_budget)
            print('Stopping...')
            break;
        if verbose:
            print('Time: %.2fs'%(time.time()-cur_t))
            print('Full Time: %.1fs'%(time.time()-start_t))
        cur_t=time.time() 
        if verbose:
            print("Current best Epoch:   %d"%best_epoch)
    
    if epochs<epoch+1:
        print("\nMaximum number of epochs reached...")
    
    # save training course
    trainCourse={'mse':train_mse,'val_mse':val_mse,'mae':train_mae,'val_mae':val_mae}
    if not path is None:
        with open(path+'trainCourse.pkl','wb') as f:    
            pickle.dump(trainCourse, f)                 
        

    # Output best result and evaluate best model on all metrics if all datasets are given
    print("\nBest MSE on Validation set: %.4g"%best_value)
    print("At training step %d"%best_epoch)
    if not path is None:
        # saves last model and loads model from best epoch
        model.save(path+'savemodels/model_last.h5')
        model=load_model(path+'savemodels/model_best.h5')
        if full_eval_bool:
            best_model_result,pred_val,pred_nest=full_eval(model,x,x_val,y,y_val,in_nest.values,in_base.values, 
                                                           res_nest[result_param].values,res_base[result_param].values,
                                                           results_scale[result_param],verbose=False)
            best_model_result.to_csv(path+'bestmodel_result.csv')
        
    print("Training ended after %.1fs"%(time.time()-start_t))
    time_per_epoch=(time.time()-start_t)/(epoch+1)
    
    # if the model could not be evaluated on the other sets return a dummy
    if not full_eval_bool:
        best_model_result=None
        
    return trainCourse,model,time_per_epoch,best_model_result

Here the predictions are evaluated.

In [74]:
def evaluate_predictions(y,y_pred,results_scale):

    # mkld needs the scaled values between 0 and 1 
    res_max=results_scale.loc['max']
    res_min=results_scale.loc['min']
    
    y=y*(res_max-res_min)+res_min
    y_pred=y_pred*(res_max-res_min)+res_min

    mse=np_mse(y,y_pred)
    mae=np_mae(y,y_pred)
    me=np_me(y,y_pred)
    perc_mse_val=perc_mse(y,y_pred)
    perc_mae_val=perc_mae(y,y_pred)
    perc_me_val=perc_me(y,y_pred)

    print("MSE:\t\t%.5g"%mse)
    print("MAE:\t\t%.5g"%mae)
    print("ME:\t\t%.5g"%me)
    print("Perc ME:\t%.5g"%perc_me_val)
    
    result={'mse':mse,'mae':mae,'me':me,'perc_mse':perc_mse_val,'perc_mae':perc_mae_val,'perc_me':perc_me_val}

    return result

        
def full_eval(model,in_train,in_val,res_train,res_val,in_nest,in_base,res_nest, res_base,results_scale,verbose=False):
    '''
    Evaluates the model on all parameters and returns pandas dataframe with the results.
    '''

    # save results in dataframe
    result=pd.DataFrame()
    
    y_pred_train=np.squeeze(model.predict(in_train))
    print("\nTraining Set:")
    res_train=evaluate_predictions(res_train,y_pred_train,results_scale)
    for key in res_train:
        result.loc[0,'train_'+key]=res_train[key]
        
    y_pred_val=np.squeeze(model.predict(in_val)) 
    print("\nValidation Set:")     
    res_val=evaluate_predictions(res_val,y_pred_val,results_scale)
    for key in res_val:
        result.loc[0,'val_'+key]=res_val[key]

    y_pred_nest=np.squeeze(model.predict(in_nest)) 
    print("\nNested Set:")
    res_nest=evaluate_predictions(res_nest,y_pred_nest,results_scale)
    for key in res_nest:
        result.loc[0,'nest_'+key]=res_nest[key]
    
    y_pred_base=model.predict(in_base)
    print("\nBase Scenario:")
    res_base=evaluate_predictions(res_base,y_pred_base,results_scale)
    for key in res_base:
        result.loc[0,'base_'+key]=res_base[key]

    # rescale data for overview file
    y_pred_val_scaled_back=scale_back(y_pred_val, results_scale)
    y_pred_nest_scaled_back=scale_back(y_pred_nest, results_scale)

    return result,y_pred_val_scaled_back,y_pred_nest_scaled_back    


def np_mse(y_true,y_pred):
    '''
    Returns the mean squared error for 2 numpy arrays.
    '''
    mse=0.
    
    for i in range(len(y_true)):
        mse=mse+(y_true[i]-y_pred[i])**2
    return mse/len(y_true)

def np_mae(y_true,y_pred):
    '''
    Returns the mean absolute error for 2 numpy arrays.
    '''
    ae=abs(y_pred-y_true)
    return ae.mean()

def np_me(y_true,y_pred):
    '''
    Returns the mean error for 2 numpy arrays.
    '''
    ae=y_pred-y_true
    return ae.mean()

def perc_me(y_true,y_pred):
    '''
    Percentage form of the absolute error. 
    '''
    p_me=(y_pred-y_true)/y_true
    return p_me.mean()*100.

def perc_mae(y_true,y_pred):
    '''
    Percentage form of the mean absolute error. 
    '''
    ae=abs(y_pred-y_true)/y_true
    return ae.mean()*100.

def perc_mse(y_true,y_pred):
    '''
    Percentage form of the mean squared error. 
    '''
    mse=0.
    
    for i in range(len(y_true)):
        mse=mse+((y_true[i]-y_pred[i])/y_true[i])**2
    return 100.*mse/len(y_true)


Here the model is evaluated on given data.

In [75]:
def evaluate(model,train_inputs,train_results,val_inputs,val_results,nest_inputs,nest_results,base_inputs,
                      base_results,inputs_scale,results_scale):

    output='o1'
    result,pred_val,pred_nest=full_eval(model,train_inputs.values,
                                     val_inputs.values,
                                     train_results[output].values,
                                     val_results[output].values,
                                     nest_inputs.values,
                                     base_inputs.values,
                                     nest_results[output].values, 
                                     base_results[output].values,
                                     results_scale[output],
                                     verbose=True)
            
    return result,pred_val,pred_nest

Read the setup file and run the training with the hyperparameters from the setup file and perform the training. 

In [76]:
import ast
import keras.backend as K

def run_training(save_path,setup_path,data_path,early_stopping=True,early_stopping_no=200):
    
    '''
    save_path [string]  Path for saving the results.
    
    setup_path [string] Path to setup_NN.csv (contains hyperparameters).
    
    data_path [string]  Path to folder containing the data.   
                
    early_stopping [boolean] default true 
                        True if you want to stop early.
                
    early_stopping_no [int] default 200 
                        Number of epochs after which you want to stop if no improvement.       
                 
    '''
    
    # assert save_path is a string and the path does not already exist
    if not save_path is None:
        assert type(save_path)==str, "save_path has to be a string"
        if not save_path[-1]=='/':
            save_path=save_path+'/'
    
    if os.path.exists(save_path):
        if os.path.exists(save_path+'overview.csv'):
            overview=pd.read_csv(save_path+'overview.csv',index_col=0)
            assert not os.path.exists(save_path+'Model0'),"Incomplete Training run in folder. Delete "+save_path+'Model0'
    else:
        os.mkdir(save_path)
        
    # assert data_path is a string and the required files do exist
    assert type(data_path)==str, "data_path has to be a string"
    if not data_path[-1]=='/':
        data_path=data_path+'/'
    
    # assert setup file exists
    assert type(setup_path)==str, "setup_path needs to be a string"
    assert os.path.exists(setup_path), "Setup "+setup_path+" does NOT exist"
          
    setup=pd.read_csv(setup_path)
        
    # convert strings from .csv file to list
    for column in setup.columns:
        setup.loc[0,column]=ast.literal_eval(setup.loc[0,column])
    
    # set time budget per evaluation in s
    timebudget_per_eval=60*60
    
    # save the setup/searchspace
    if not save_path is None:
        setup.to_csv(save_path+'current_setup.csv',index=False)
        
    # read from setup file
    batchsize = (setup.loc[0,'batchsize'])[0]
    no_layers = (setup.loc[0,'no_layers'])[0]
    dropouts = setup.loc[0,'dropouts']
    no_nodes = setup.loc[0,'no_nodes']
    activation = setup.loc[0,'activation']
    initializer = setup.loc[0,'initializer']
    optimizer = setup.loc[0,'optimizer']
    lr = (setup.loc[0,'learning_rate'])[0]
    output_activation = setup.loc[0,'output_activation_function']
        
    # get data
    x,y,x_val,y_val,in_nest,res_nest,in_base,res_base,inputs_scale,results_scale=get_data(data_path)
    
    
    input_dim=x.shape[1]
    # set random seed otherwise everytime it is reset the same seed is used 
    # because numpy gets seeded in data.get_train_data
    
    # make overview file
    overview=pd.DataFrame(columns=['no_layers','no_nodes','dropouts','batchsize','activation','initializer',
                                    'optimizer','timebudget_per_eval','no_epochs','time_per_epoch','train_mse',
                                    'val_mse','lr','early_stopping','early_stopping_no','output_activation_function'])
       
    start_t=time.time()
        
    model_name='Model0'
    if not save_path is None:
        save_path_model=save_path+model_name
            
    if not lr is None:
        if optimizer=='RMSprop':
            optimizer=kOpt.RMSprop(lr=lr)
        elif optimizer=='adam':
            optimizer=kOpt.Adam(lr=lr)
                    
        overview.loc[0,'no_layers']=no_layers
        overview.at[0,'no_nodes']=no_nodes
        overview.at[0,'dropouts']=dropouts
        overview.loc[0,'batchsize']=batchsize
        overview.loc[0,'activation']=activation
        overview.loc[0,'initializer']=initializer
        overview.loc[0,'optimizer']=type(optimizer).__name__
        overview.loc[0,'timebudget_per_eval']=timebudget_per_eval
        overview.loc[0,'lr']=lr
        overview.loc[0,'early_stopping']=early_stopping
        overview.loc[0,'early_stopping_no']=early_stopping_no
        overview.loc[0,'output_activation_function']=output_activation
            
        model=setup_model(hidden_layers=no_nodes,dropouts=dropouts,activation=activation,input_dim=input_dim,
                                initializer=initializer,optimizer=optimizer,output_activation=output_activation)
    
        trainCourse,model,time_epoch,best_res=train_model(model,x,y,x_val,
                                                   y_val,epochs=2000,
                                                   results_scale=results_scale,
                                                   batchsize=batchsize,
                                                   early_stopping = early_stopping,
                                                   early_stopping_no=early_stopping_no,
                                                   path=save_path_model,
                                                   time_budget=timebudget_per_eval,
                                                   in_nest=in_nest,
                                                   in_hval=x_val,
                                                   in_base=in_base,
                                                   res_nest=res_nest,
                                                   res_hval=y_val,
                                                   res_base=res_base,
                                                   val_metric='mae')
            
        # find best epoch
        for column in best_res.columns:
            overview.loc[0,column]=best_res.loc[0,column]
                
        overview.loc[0,'no_epochs'] = len(trainCourse['mse'])
        overview.loc[0,'time_per_epoch'] = time_epoch
    
        if not save_path is None:
            overview.to_csv(save_path+'/overview.csv')
                
        print("\n\nTotal Time: %.1fs"%(time.time()-start_t))
            
    return overview,model

## Polynomial

In this section we perform a regression using a given polynomial term structure. A more sophisticated methods using an adaptive stepwise approach have been implemented in the industry, see Krah, Nikolić, Korn (2018).

We read in the data without scaling it. Please note that the unscaled data are read in once again here. This was done for the sake of convenience and it allows a separate run of polynomial regression without a training of the neural network.

In [77]:
def get_data_unscaled(path):
    
    # assert path is a string and the required files do exist
    assert type(path)==str, "path has to be a string"
    if not path[-1]=='/':
        path=path+'/'
    
    assert os.path.exists(path+'train_input.csv'), "File '"+ path+"train_input.csv' is missing"
    assert os.path.exists(path+'validation_input.csv'), "File '"+ path+"validation_input.csv' is missing"
    assert os.path.exists(path+'nested_input.csv'), "File '"+ path+"nested_input.csv' is missing"
    assert os.path.exists(path+'base_input.csv'), "File '"+ path+"base_input.csv' is missing"
    assert os.path.exists(path+'train_result.csv'), "File '"+ path+"train_result.csv' is missing"
    assert os.path.exists(path+'validation_result.csv'), "File '"+ path+"validation_result.csv' is missing"
    assert os.path.exists(path+'nested_result.csv'), "File '"+ path+"nested_result.csv' is missing"
    assert os.path.exists(path+'base_result.csv'), "File '"+ path+"base_result.csv' is missing"
    
        
    # =============================================================================
    #     Specify paths
    # =============================================================================
        
    # train paths
    path_train_res=path+'train_result.csv'
    path_train_input=path+'train_input.csv'
             
    # validation paths  
    path_val_res=path+'validation_result.csv'
    path_val_input=path+'validation_input.csv'
        
    # nested paths
    path_nest_res=path+'nested_result.csv'
    path_nest_input=path+'nested_input.csv'
        
    # base paths
    path_base_res=path+'base_result.csv'
    path_base_input=path+'base_input.csv'

    
    # =============================================================================
    #   load data
    # =============================================================================
        
    # train set
    train_results=pd.read_csv(path_train_res,index_col=0)
    train_inputs=pd.read_csv(path_train_input,index_col=0)
    
    # validation set
    val_results=pd.read_csv(path_val_res,index_col=0)
    val_inputs=pd.read_csv(path_val_input,index_col=0)
    
    # nested set
    nest_results=pd.read_csv(path_nest_res,index_col=0)
    nest_inputs=pd.read_csv(path_nest_input,index_col=0)
    
    # base set
    base_results=pd.read_csv(path_base_res,index_col=0)
    base_results=base_results.iloc[[0],:]
    base_inputs=pd.read_csv(path_base_input,index_col=0)
   
   
    return train_inputs, train_results, val_inputs, val_results, nest_inputs, nest_results

Convert a candidate vector to the corresponding data column. The terminology "candidate" comes from a wrapper which iteratively builds up a model using an information criterion. Here we provide a stripped up code as a minimum viable regression algorithm.

In [78]:
def candidate_to_vector(c, X):
    
    vec = 1
    c_str = ''
    
    for i in range(X.shape[1]):
        vec = vec * X[:,i]**c[i]                           
        if c[i]>0:
            # Generates a String for the output
            c_str = c_str + 'i' + str(i+1)                   
            if c[i]>1:
                c_str = c_str + '^' + str(int(c[i]))
            c_str = c_str + ' '        
            
    return (np.transpose(np.array([vec])), c_str)          

Run the least-squares regression. Setup file which contains the polynomial that is used in the regression is named 'setup_polynomial.csv'. We provide a version with just linear terms. Of course more complex models can easily be used.

In [79]:
from sklearn.linear_model import LinearRegression
from numpy import genfromtxt

def export_results(data_path,result_param='o1'):
   
    # load data
    train_inputs, train_results, scenarios_t, results_t,scenarios_nest,results_nest =get_data_unscaled(data_path)
    
    # load polynomial 
    model =  genfromtxt('./setups/setup_polynomial.csv', delimiter=',')
    model = model[1:model.shape[0],1:model.shape[1]]

    # ================================= #
    # Training                          #
    # ================================= #
    
    # calculate X for validation data
    results= train_results[result_param]
    inputs = np.ones([train_inputs.shape[0],1]).astype(float)
    for i in (range(model.shape[0]-1)):
        v, v_text= candidate_to_vector(model[i+1,:],train_inputs.values)
        inputs = np.float64(np.append(arr=inputs, values=v, axis=1))
        
    # calculate regression coefficients
    reg = LinearRegression().fit(inputs, results)    
    
    
    # ================================== #
    #   Validation Data                  #
    # ================================== #

    inputs_val = np.ones([scenarios_t.shape[0],1]).astype(float)
    for i in (range(model.shape[0]-1)):
        v, v_text = candidate_to_vector(model[i+1,:],scenarios_t.values)
        inputs_val = np.float64(np.append(arr=inputs_val, values=v, axis=1))
    # calculated y_pred for validation
    y_pred_t = np.float64(reg.predict(inputs_val))
        
    # ================================= #
    #   Nested data                     #
    # ================================= #
    
    ## calculate X for nested data
    inputs_nest = np.ones([scenarios_nest.shape[0],1]).astype(float)
    for i in (range(model.shape[0]-1)):
        v, v_text = candidate_to_vector(model[i+1,:],scenarios_nest.values)
        inputs_nest = np.float64(np.append(arr=inputs_nest, values=v, axis=1))
    
    ## calculate regression coefficients   
    #reg = LinearRegression().fit(X_nest, result_nest)
    # calculated y_pred for nested data
    y_pred_nest = np.float64(reg.predict(inputs_nest))

    return(y_pred_t, y_pred_nest, model, reg.coef_)

Write overview file with inputs, output "o1" and predictions from the neural network and the polynomial for validation and nested data.

In [80]:
def write_overview(data_path,save_path):
    
    # =============================================================================
    #     Specify paths
    # =============================================================================
                 
    # validation set
    path_val_input=data_path+'validation_input.csv'
    path_val_res=data_path+'validation_result.csv'
    
    # nested set
    path_nest_input=data_path+'nested_input.csv'
    path_nest_res=data_path+'nested_result.csv'
    
    # =============================================================================
    #   load data
    # =============================================================================

    # validation set
    val_results=pd.read_csv(path_val_res,index_col=0)
    val_inputs=pd.read_csv(path_val_input,index_col=0)
        
    # nested set
    nest_results=pd.read_csv(path_nest_res,index_col=0)
    nest_inputs=pd.read_csv(path_nest_input,index_col=0)

    
    # =============================================================================
    # write file
    # =============================================================================

    dim_val=val_inputs.shape[0]
    inputs=val_inputs.shape[1]
    dim_nest=nest_inputs.shape[0]
    
    # validation data
    validation=pd.DataFrame(columns=val_inputs.columns)
    
    for i in range(inputs):
        for j in range(dim_val):
            validation.loc[j+1,'i'+str(i+1)] = val_inputs.loc[j+1,'i'+str(i+1)]
    
    validation[''] = ''
    validation['o1'] = val_results['o1']

    # prediction from neural network
    path_pred_val=save_path+'/Neural_Network/prediction_val_nn.csv'
    prediction_val=pd.read_csv(path_pred_val,index_col=0)
    prediction_val.index = prediction_val.index + 1
    validation.insert(inputs+2, 'Prediction NN ',prediction_val, True) 
    
    # prediction from polynomial
    prediction_val_poly = pd.read_csv(save_path+'/Polynomial/prediction_val_poly.csv',index_col=0)
    prediction_val_poly.index = prediction_val_poly.index + 1
    validation.insert(inputs+3, 'Polynomial', prediction_val_poly)    
    
    validation.to_csv(save_path+'/overview_validation.csv') 


    # nested data
    nested=pd.DataFrame(columns=nest_inputs.columns)
    
    for i in range(inputs):
        for j in range(dim_nest):
            nested.loc[j+1,'i'+str(i+1)] = nest_inputs.loc[j+1,'i'+str(i+1)]
            
    nested[''] = ''
    nested['o1'] = nest_results['o1']
    
    # prediction from neural network
    path_pred_nest=save_path+'/Neural_Network/prediction_nest_nn.csv'
    prediction_nest=pd.read_csv(path_pred_nest,index_col=0)
    prediction_nest.index = prediction_nest.index + 1
    nested.insert(inputs+2, 'Prediction NN ',prediction_nest, True) 
  
    # prediction from polynomial
    prediction_nest_poly = pd.read_csv(save_path+'/Polynomial/prediction_nest_poly.csv',index_col=0)
    prediction_nest_poly.index = prediction_nest_poly.index + 1
    nested.insert(inputs+3,'Polynomial', prediction_nest_poly, True)  

    nested.to_csv(save_path+'/overview_nested.csv') 

# Run the Neural Network and the Polynomial

Here we actually run the code and save the predictions in the overview file.

In order to run the code one needs to choose portfolio 1, 2 or 3. The results will be saved in './results/', e.g. save_path = './results/Portfolio_1_TIMESTAMP'.

Input data will be read for the correct company, e.g. data_path = './Portfolio1/'.

Setup file for neural network contains hyperparameters, it has to be named 'setup_NN.csv'. Parameters to be set in setup_NN.csv:
- batchsize
- no_layers
- dropouts
- no_nodes
- activation
- initializer
- optimizer
- learning_rate
- output_activation_function

Setup file for polynomial contains the polynomial term structure that is used in regression, it has to be named 'setup_polynomial.csv'.

In [81]:
import warnings
import xlsxwriter

warnings.filterwarnings("ignore", category=FutureWarning)

# INPUT: Portfolio number #
portfolio = '1'           
###########################

data_path = './Portfolio'+portfolio+'/'

today=datetime.now()
timestamp=today.strftime("%Y_%b_%d_%H-%M-%S")

save_path = './results/Portfolio_'+portfolio+'_'+timestamp

assert not os.path.exists(save_path), "Folder '"+ save_path +"' already exists"
os.makedirs(save_path)


# =============================================================================
# NEURAL NETWORK
# =============================================================================

setup_path_NN = './setups/setup_NN.csv'
save_path_NN = save_path+'/Neural_Network'

'''
set (if desired to be different from default)
- early_stopping (default = True)
- early_stopping_no (default = 200)
'''
early_stopping=True
early_stopping_no=5

# Run neural network training
overview,model=run_training(save_path_NN,setup_path_NN,data_path,early_stopping=early_stopping,
                            early_stopping_no=early_stopping_no)

# Load data
train_inputs,train_results,val_inputs,val_results,nest_inputs,nest_results,base_inputs,base_results,inputs_scale,results_scale=get_data(data_path)

# Evaluate model
result,pred_val_NN,pred_nest_NN=evaluate(model,train_inputs,train_results,val_inputs,val_results,
                      nest_inputs,nest_results,base_inputs,base_results,inputs_scale,results_scale)
result.to_csv(save_path_NN+'/results.csv')

# Save predictions
prediction_val_NN=pd.DataFrame({'pred_val':pred_val_NN})
prediction_val_NN.to_csv(save_path_NN+'/prediction_val_nn.csv') 

prediction_nest_NN=pd.DataFrame({'pred_nest':pred_nest_NN})
prediction_nest_NN.to_csv(save_path_NN+'/prediction_nest_nn.csv') 

K.clear_session()


# =============================================================================
# Polynomial
# =============================================================================

save_path_poly = save_path+'/Polynomial/'
os.mkdir(save_path_poly)

# run regression and get results
pred_val_poly,pred_nest_poly, model, coef = export_results(data_path)

prediction_val_poly=pd.DataFrame({'pred_val':pred_val_poly})
prediction_val_poly.to_csv(save_path_poly+'/prediction_val_poly.csv') 

prediction_nest_poly=pd.DataFrame({'pred_nest':pred_nest_poly})
prediction_nest_poly.to_csv(save_path_poly+'/prediction_nest_poly.csv') 

xlsx_name = 'Polynomial_Coefficients.xlsx'

workbook = xlsxwriter.Workbook(save_path_poly + xlsx_name)
worksheet = workbook.add_worksheet("Coefficients")
row = 1
col = 1
for j in range(model.shape[1]):
    worksheet.write(0, col+j, 'i'+str(j+1))
for i in range(model.shape[0]):
    for j in range(model.shape[1]):
        worksheet.write(i+1, j+1, model[i][j])
for i in range(len(coef)):
    worksheet.write(i+1,0,coef[i])
worksheet.write(0,0,'Coefficient')
workbook.close()



# ================================================================================================
#  Write overview file with inputs, output o1 and prediction from neural network and polynomial
# ================================================================================================

write_overview(data_path,save_path)
print('\nOverview files can be found in',save_path)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 13)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                700       
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 70)                3570      
_________________________________________________________________
dropout_2 (Dropout)          (None, 70)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 50)                3550      
_________________________________________________________________
dropout_3 (Dropout)          (None, 50)                0   